In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


phase_counts = df.groupby('Athlete')['COVID_STATUS'].nunique()
eligible_athletes = phase_counts[phase_counts >= 2].index
df_filtered = df[df['Athlete'].isin(eligible_athletes)]


athlete_venues = df_filtered.groupby(['Athlete', 'COVID_STATUS'])['Venue_Type'].apply(set).unstack()
athlete_venues = athlete_venues.applymap(lambda x: x if isinstance(x, set) else set())
print(athlete_venues.columns)

# Συνάρτηση ανίχνευσης μετακίνησης
def detect_shift_v3(row):
    before = row.get('Before COVID', set())
    during = row.get('During COVID', set())
    after = row.get('After COVID', set())

    active_phases = sum([bool(before), bool(during), bool(after)])
    if active_phases < 2:
        return 'No Data'

    all_venues = list(before) + list(during) + list(after)

    if 'Major' in all_venues and 'Non-Major' in all_venues:
        if 'Major' in before and 'Non-Major' in after:
            return 'Major to Non-Major'
        elif 'Non-Major' in before and 'Major' in after:
            return 'Non-Major to Major'
        else:
            return 'Mixed'
    elif set(all_venues) == {'Major'}:
        return 'Stayed Major'
    elif set(all_venues) == {'Non-Major'}:
        return 'Stayed Non-Major'
    else:
        return 'Mixed'


athlete_venues['Shift'] = athlete_venues.apply(detect_shift_v3, axis=1)


print(" Πίνακας Αθλητών με Περιόδους και Venue Types:")
print(athlete_venues[['Before COVID', 'During COVID', 'After COVID', 'Shift']].head(10))

print("Σύνοψη Τάσεων Μετακίνησης:")
print(athlete_venues['Shift'].value_counts())

import seaborn as sns


plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")  


custom_colors = {
    'Major to Non-Major': '#FF6F61',
    'Non-Major to Major': '#6B5B95',
    'Stayed Major': '#88B04B',
    'Stayed Non-Major': '#F7CAC9',
    'Mixed': '#92A8D1',
    'No Data': '#d3d3d3'
}


shift_counts = athlete_venues['Shift'].value_counts()

sns.barplot(
    x=shift_counts.index,
    y=shift_counts.values,
    palette=[custom_colors.get(shift, '#333333') for shift in shift_counts.index]
)

plt.title(" Τάσεις Μετακίνησης Αθλητών (Before / During / After COVID)", fontsize=14, weight='bold')
plt.ylabel("Αριθμός Αθλητών", fontsize=12)
plt.xlabel("Τύπος Μετακίνησης", fontsize=12)
plt.xticks(rotation=25, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()

